In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
plt.style.use('ggplot')
plt.style.use(['dark_background'])

https://towardsdatascience.com/8-popular-sql-window-functions-replicated-in-python-e17e6b34d5d7

# RANK, DENSE_RANK and ROW_NUMBER
https://codingsight.com/similarities-and-differences-among-rank-dense_rank-and-row_number-functions/

The RANK, DENSE_RANK and ROW_NUMBER functions are used to `retrieve an increasing integer value`. They start with a value based on the condition imposed by the ORDER BY clause. All of these functions `require the ORDER BY` clause to function properly. In case of partitioned data, the integer counter is `reset to 1 for each partition`.

The DENSE_RANK function is similar to RANK function however the DENSE_RANK function does not skip any ranks if there is a tie between the ranks of the preceding records. 
- Eg. the first two values are the same and the third different, (1,1,2) is returned instead of (1,1,3) like in Rank

Unlike the RANK and DENSE_RANK functions, the ROW_NUMBER function simply returns the row number of the sorted records starting with 1.

> If no duplicated values in ORDER BY clause, they are equivalent

## row number => rank(method='first')

> Note: In pandas-- the partition is is the groupby field and the col_selected is the order by field

The SQL Row Number() function, assigns a sequential integer to each row within the partition of a dataset. It is often used in practice to create an auxiliary column that ranks a field based on the specified partition and order. The column can then be used to more effectively filter or join the dataset.

```row number() over(partition by customer_id order by order_date)```

With pandas the same result can be achieved by applying the .rank(method = ‘first’) function to a GroupBy object filtered by Order Date to create the Row Num column:

```
orders['Row Num'] = orders.groupby(['Customer ID'])['Order Date'].rank(method='first') 
```

To verify, let's have the df display it

```
orders.sort_values(by= ['Customer ID','Order Date'], inplace = True)
orders = orders.reset_index(drop = True)
orders.head(20)
```

> Note 1: if you wanted the opposite, just set ascending to be False in rank

```orders.groupby([‘Customer ID’])[‘Order Date’].rank(method=’first’, ascending = False)```

> Note 2: paritition by multiple columns just need to add the preferred fields to the GroupBy object 

```
#SQL Syntax
row number() over(partition by customer_id, order_month order by order_date)

#Python Syntax
orders.groupby([‘Customer ID’, 'Order Month'])[‘Order Date’].rank(method=’first')
```

## rank() → rank(method=’min’)

> Note the main difference between this and row number is that rank makes duplicates where as row number does not

## Dense_rank() → Rank(method=’dense’)

##  first/last value() → rank(method =’first’, ascending = True/False) == 1

The first value() and last value() window functions are used to retrieve the value of the first or last row, in an ordered partition of a dataset.

```
#The aapl dataset includes prices for one stock only, therefore the #partition is just applied on order_month
first value(adj_close) over(partition by order_month order by date)
last value(adj_close) over(partition by order_month order by date)
```

To make this work in pandas, he uses rank again (method='fist') but to make it so that they are one to one with num_rows he bfills and ffills (otherwise he just flips ascending around)

```
aapl['First Value'] = aapl['Adj Close'].loc[aapl.groupby('Month')['Date'].rank(method ='first') == 1]
aapl['First Value'] = aapl['First Value'].bfill()

aapl['Last Value'] = aapl['Adj Close'].loc[aapl.groupby('Month')['Date'].rank(method ='first', ascending = False) == 1]
aapl['Last Value'] = aapl['Last Value'].ffill()
```


# Rolling sums and averages

Note that we may want to do it by partition (he doesn, over the last n timesteps, order them by another column

## Sum(…) over(partition by … order by .. rows unbounded preceding) → cumsum()

The syntax below is not exactly the most efficient and intuitive: there is quite a lot of code to write and if we wish to sort by order_date, we must also include the rows… clause to compute a cumulative sum instead of a total sum in the chosen window.

```
sum(amount_paid) over(partition by customer_id, order_month order by order_date rows unbounded preceding)
```

Again, partition becomes groupby, col to be summed on is selected (I believe if we wanted it ordered by it would have to be sorted first)

```
orders["Run Tot (£)"] = orders.groupby(['Customer ID', 'Order Month'])['Amount Paid (£)'].cumsum()
```

## Avg(…) over(partition by… ) → transform(np.mean)
In a similar fashion, we may also wish to compute the average amount spent by each customer per month. This time the SQL syntax is quite intuitive:

```
orders["Run Avg (£)"] = orders.groupby(['Customer ID', 'Order Month'])['Amount Paid (£)'].transform('mean').round(1)
```

The reason why we used pandas DataFrame.transform() function is that it allows us to keep the length of the Series equal to the length of the original dataset after computing the mean on the GroupBy object

```
orders["Run Avg (£)"] = orders.groupby(['Customer ID', 'Order Month'])['Amount Paid (£)'].transform('mean').round(1)
```

> Note: to order by you need to sort the df first

## means and sums over window lengths
### Avg(…) over(partition by … order by .. rows between n preceding and current row) → rolling(window = …).mean()

```
avg(adj_close) over(order by date rows between 6 preceding and current row)
```

because you want to do it over a range you use rolling

```

aapl.sort_values(by= 'Date', ascending = True, inplace = True)

aapl['Rolling Mean 7d'] = aapl['Adj Close'].rolling(7).mean().round(1)
aapl['Expanding Mean 7d'] = aapl['Adj Close'].expanding().mean().round(1)
```


# Lead/Lag
Moving a col up and down based on a value desired

> `shift(n)` if n>0 then it shifts it up, if n<0 it shifts it down